In [1]:
import os
os.path.exists("/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D")

True

In [2]:
path = "/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D"

print(os.listdir(path))

['cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5', 'cam4DLC_resnet152_CheetahOct14shuffle1_500000.h5', 'cam5DLC_resnet152_CheetahOct14shuffle1_500000.h5', 'cam6DLC_resnet152_CheetahOct14shuffle1_500000.h5']


In [3]:
for root, dirs, files in os.walk(path):
    level = root.replace(path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

filtered_2D/
    cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5
    cam4DLC_resnet152_CheetahOct14shuffle1_500000.h5
    cam5DLC_resnet152_CheetahOct14shuffle1_500000.h5
    cam6DLC_resnet152_CheetahOct14shuffle1_500000.h5


In [4]:
import pandas as pd

file_path = "/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5"

df = pd.read_hdf(file_path)

df.head()

scorer    DLC_resnet152_CheetahOct14shuffle1_500000                         \
bodyparts                                     r_eye                          
coords                                            x           y likelihood   
0                                        466.780748  496.310020   0.843646   
1                                        476.662960  496.780764   0.867980   
2                                        467.291212  496.760568   0.816340   
3                                        467.653452  496.879998   0.833451   
4                                        476.905672  496.717315   0.845231   

scorer                                                                \
bodyparts       l_eye                         r_shoulder               
coords              x           y likelihood           x           y   
0          477.761635  497.109234   0.865188  468.364375  505.838816   
1          477.798796  497.308653   0.812952  467.206840  513.339016   
2          477.738068  497.299124   0.826227  467.445744  513.202022   
3          477.482832  497.376827   0.834253  467.456618  505.628165   
4          477.585308  497.492764   0.901569  478.050518  513.924471   

scorer                             ...                                     \
bodyparts            r_front_knee  ...       lure   tail_base               
coords    likelihood            x  ... likelihood           x           y   
0           0.648893   477.975610  ...   0.099997  458.291532  496.340185   
1           0.706509   467.915626  ...   0.048126  458.457967  496.305668   
2           0.631165   467.901369  ...   0.060170  458.221192  496.353792   
3           0.480500   467.914589  ...   0.075540  467.733226  496.262832   
4           0.476722   467.821035  ...   0.070295  467.384385  496.280282   

scorer                                                               \
bodyparts                   nose                          neck_base   
coords    likelihood           x           y likelihood           x   
0           0.779278  467.133634  504.481726   0.874825  477.818254   
1           0.708053  478.029599  505.899045   0.845803         NaN   
2           0.559946  467.599322  505.047461   0.829336         NaN   
3           0.499875  467.399360  505.536394   0.850702         NaN   
4           0.394748  477.986238  506.087321   0.885310         NaN   

scorer                            
bodyparts                         
coords              y likelihood  
0          497.394294   0.306026  
1                 NaN   0.294876  
2                 NaN   0.276134  
3                 NaN   0.253217  
4                 NaN   0.273809  

[5 rows x 75 columns]

In [14]:
import json
import numpy as np
from pathlib import Path

def load_scene(scene_json_path: Path) -> list:
    with open(scene_json_path, "r") as f:
        data = json.load(f)

    cameras = []

    for cam in data["cameras"]:
        # Intrinsics
        K = np.array(cam["k"], dtype=np.float64).reshape(3, 3)

        # Distortion (not needed for DLT, but useful to keep)
        D = np.array(cam["d"], dtype=np.float64)

        # Extrinsics
        R = np.array(cam["r"], dtype=np.float64).reshape(3, 3)
        t = np.array(cam["t"], dtype=np.float64).reshape(3, 1)

        # Projection matrix: P = K [R | t]
        Rt = np.hstack([R, t])   # 3x4
        P = K @ Rt               # 3x4

        cameras.append({
            "K": K,
            "D": D,
            "R": R,
            "t": t,
            "P": P
        })

    return cameras

In [15]:
cam_path = Path("/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/extrinsic_calib/4_cam_scene_sba.json")

cameras = load_scene(cam_path)

In [16]:
P_matrices = [cam["P"] for cam in cameras]

# Check shapes
for i, P in enumerate(P_matrices):
    print(f"Camera {i+1} projection matrix shape:", P.shape)

Camera 1 projection matrix shape: (3, 4)
Camera 2 projection matrix shape: (3, 4)
Camera 3 projection matrix shape: (3, 4)
Camera 4 projection matrix shape: (3, 4)


In [17]:
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional

def _find_column_triplet(df: pd.DataFrame, bodypart: str) -> Tuple[Tuple, Tuple, Tuple]:
    """
    Find the (x,y,likelihood) column tuples for a given bodypart in a DLC-style MultiIndex df.
    Returns (xcol, ycol, lcol) as column-key tuples suitable for df[...] indexing.
    Raises KeyError if not found.
    """
    # Accept both 2- or 3-level column index shapes, but DLC usually has 3-level: (scorer, bodypart, coord)
    cols = list(df.columns)
    xcol = ycol = lcol = None
    for c in cols:
        # treat c as tuple-like
        if len(c) >= 2 and c[1] == bodypart:
            coord = c[-1]  # assume final level is 'x','y','likelihood'
            if coord == 'x':
                xcol = c
            elif coord == 'y':
                ycol = c
            elif coord in ('likelihood','prob','score','p'):
                lcol = c
    if xcol is None or ycol is None:
        raise KeyError(f"couldn't find x/y columns for bodypart '{bodypart}'")
    # if likelihood missing, create a dummy column tuple (we'll fill with NaNs later)
    if lcol is None:
        # create a fake tuple with 'likelihood' appended (will error if used for indexing)
        # We prefer returning None to indicate missing
        lcol = None
    return xcol, ycol, lcol

def get_joint_from_df(df: pd.DataFrame, bodypart: str, frame: int) -> Tuple[Optional[float], Optional[float], Optional[float]]:
    """
    Return (u, v, likelihood) for a given bodypart at a specific frame index from a single camera DataFrame.
    If likelihood column is missing, returns np.nan for likelihood.
    """
    try:
        xcol, ycol, lcol = _find_column_triplet(df, bodypart)
    except KeyError:
        return (np.nan, np.nan, np.nan)
    u = df.at[frame, xcol] if xcol in df.columns else np.nan
    v = df.at[frame, ycol] if ycol in df.columns else np.nan
    if lcol is not None and lcol in df.columns:
        l = df.at[frame, lcol]
    else:
        l = np.nan
    # ensure numeric (could be pandas Series scalar)
    return float(u) if pd.notna(u) else np.nan, float(v) if pd.notna(v) else np.nan, float(l) if pd.notna(l) else np.nan

def triangulate_point_dlt(us_vs: List[Tuple[float,float]], Ps: List[np.ndarray]) -> Optional[np.ndarray]:
    """
    us_vs: list of (u,v) observations (pixel coords) for the same 3D point across cameras
    Ps: corresponding list of 3x4 camera projection matrices (same order)
    Returns: 3-element np.array [X, Y, Z] (float) on success, or None if failure.
    """
    if len(us_vs) < 2:
        return None

    # Build A (2*C x 4)
    A_rows = []
    for (u, v), P in zip(us_vs, Ps):
        p1 = P[0, :]  # 1x4
        p2 = P[1, :]
        p3 = P[2, :]
        A_rows.append(u * p3 - p1)
        A_rows.append(v * p3 - p2)
    A = np.vstack(A_rows)  # shape (2C, 4)

    # Solve by SVD: A x = 0 -> x is last col of V (Vt[-1])
    try:
        _, _, Vt = np.linalg.svd(A)
    except np.linalg.LinAlgError:
        return None
    X = Vt[-1, :]  # last row of Vt is last column of V
    if np.isclose(X[-1], 0):
        # cannot homogenize; ill-conditioned
        return None
    X = X / X[-1]
    return X[:3]  # X, Y, Z

def triangulate_frame_all_joints(
    cams_dfs: List[pd.DataFrame],
    P_matrices: List[np.ndarray],
    frame_idx: int,
    conf_thresh: float = 0.6,
    bodyparts: Optional[List[str]] = None
) -> Dict[str, Optional[np.ndarray]]:
    """
    Triangulate each bodypart for a single frame.
    - cams_dfs: list of pandas DataFrames, one per camera (same ordering as P_matrices)
    - P_matrices: list of 3x4 numpy arrays
    - frame_idx: integer row index in the DataFrames (same frame indexing across cams)
    - conf_thresh: min likelihood to count a detection
    - bodyparts: optional list of bodypart names to process; if None, infer from first df
    Returns: dict mapping bodypart -> np.array([X,Y,Z]) or None if triangulation not possible.
    """
    if len(cams_dfs) != len(P_matrices):
        raise ValueError("number of cameras (dfs) must match number of projection matrices")

    # infer bodyparts from first DataFrame if not supplied
    if bodyparts is None:
        first_df = cams_dfs[0]
        # find second level values assuming MultiIndex with bodypart as level 1
        if isinstance(first_df.columns, pd.MultiIndex) and first_df.columns.nlevels >= 2:
            bodyparts = list(dict.fromkeys(first_df.columns.get_level_values(1)))  # preserve order, unique
        else:
            # fallback: can't infer
            raise ValueError("Cannot infer bodyparts from the DataFrame columns. Provide bodyparts argument.")

    results = {}
    for bp in bodyparts:
        obs = []   # list of (u, v)
        Ps = []    # corresponding projection matrices
        for df, P in zip(cams_dfs, P_matrices):
            u, v, l = get_joint_from_df(df, bp, frame_idx)
            # treat NaN or small likelihood as missing
            if np.isfinite(u) and np.isfinite(v) and (np.isfinite(l) and l >= conf_thresh):
                obs.append((u, v))
                Ps.append(P)
            # if there is no likelihood column (l is NaN), you may optionally still use the point:
            # elif np.isfinite(u) and np.isfinite(v) and np.isnan(l):
            #     obs.append((u,v)); Ps.append(P)
        if len(obs) < 2:
            results[bp] = None
        else:
            X = triangulate_point_dlt(obs, Ps)
            results[bp] = X  # np.array([X,Y,Z]) or None
    return results


In [18]:
# load your 4 camera DataFrames (cam3..cam6) exactly as you already did:
cam3 = pd.read_hdf("/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5")
cam4 = pd.read_hdf("/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam4DLC_resnet152_CheetahOct14shuffle1_500000.h5")
cam5 = pd.read_hdf("/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam5DLC_resnet152_CheetahOct14shuffle1_500000.h5")
cam6 = pd.read_hdf("/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam6DLC_resnet152_CheetahOct14shuffle1_500000.h5")

cams = [cam3, cam4, cam5, cam6]

Ps = [cam["P"] for cam in cameras]

frame = 0
triangulated = triangulate_frame_all_joints(cams, Ps, frame, conf_thresh=0.6)

# inspect one joint
print("nose 3D:", triangulated.get("nose"))

nose 3D: [-10.3823387    4.64170446   0.76591981]
